# Anas Roumeih 3766647

This notebook shows a simulation of a wave packet colliding with a rectangular potential, written in python. This simulation illustrates the key quantum mechanics concepts of quantum tunneling, and reflection.

The Hamiltonian of an electron (atomic units $\hbar = m_e = 1$): 

$$ \hat{H} = -\frac{1}{2}\frac{\text{d}^2}{\text{d}x^2} + V(x) $$

Where the potential is given by:

$$ V(x) =  \left\{
\begin{array}{ll}
      V_0 & x_{min} \leq x \leq x_{max} \\
      0 & \text{else} \\
\end{array} 
\right.$$ 

The initial wave packet is given by [1]: 

$$ \psi(x) = \left( \frac{1}{2\pi \sigma_0^2} \right)^{\frac{1}{4}} \exp\left( -\frac{(x - x_0)^2}{4\sigma_0^2} \right) \exp(i k_0 x) $$

In [18]:
import matplotlib
matplotlib.use('TkAgg')
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from scipy.sparse import linalg as la
from scipy import sparse 
import matplotlib.animation as anim


In [2]:
class WavePacket:
    def __init__(self, grid_size, time_step, potential_height, potential_width, wave_number = 1, packet_width=5.0, packet_center=-50.0, x_min=-200.0, x_max=200.0):

        self.grid_size = grid_size        # Number of points in the spatial grid
        self.time_step = time_step        # Time between each update of the wave function
        self.potential_height = potential_height  # Height of the potential barrier
        self.potential_width = potential_width    # Width of the potential barrier
        self.packet_width = packet_width  # Initial width of the Gaussian wave packet
        self.wave_number = wave_number    # Initial wave number (momentum)
        self.packet_center = packet_center  # Initial center of the wave packet
        self.prob_density = np.zeros(grid_size)  # Probability density array initialization

        """ Step 1: Discretize the spatial grid """
        self.x_values, self.delta_x = np.linspace(x_min, x_max, grid_size, retstep=True)  # Set up the spatial grid

        """ Step 2: Initialize the wave function (Gaussian wave packet) """
        self.wave_function = np.exp(-(self.x_values - packet_center)**2 / (4.0 * packet_width**2)).astype(np.complex128) #Gaussian
        self.wave_function *= np.exp(1.0j * wave_number * self.x_values) #plane wave factor
        self.wave_function *= (2.0 * np.pi * packet_width**2)**(-0.25) #normalization factor

        """ Step 3: Create the potential barrier """
        self.potential_barrier = np.array([potential_height if 0.0 < x < potential_width else 0.0 for x in self.x_values])

        """ Step 4: Build the Hamiltonian matrix """
        diag_elements = np.ones(grid_size) / self.delta_x**2 + self.potential_barrier # diagonal elements
        off_diag_elements = np.ones(grid_size - 1) * (-0.5 / self.delta_x**2) #off diagonal elements
        hamiltonian_matrix = sparse.diags([diag_elements, off_diag_elements, off_diag_elements], [0, 1, -1]) #combine all elements

        """ Step 5: Calculate the Crank-Nicolson evolution matrix """
        backward_part = (sparse.eye(self.grid_size) - time_step / 2.0j * hamiltonian_matrix).tocsc() #the matrix B
        forward_part = (sparse.eye(self.grid_size) + time_step / 2.0j * hamiltonian_matrix).tocsc() #the matrix A, csc (Compressed Sparse Column) format is used for efficient matrix operations
        self.evolution_operator = la.inv(backward_part).dot(forward_part) # A inverse times B

    def evolve_wave_function(self):
        self.wave_function = self.evolution_operator.dot(self.wave_function)  # Apply the evolution matrix
        self.prob_density = abs(self.wave_function)**2  # Calculate probability density

        # Normalize the probability density and wavefunction
        total_probability = sum(self.prob_density) 
        self.prob_density /= total_probability
        self.wave_function /= total_probability**0.5

        return self.prob_density  # Return the updated probability density

In the code, the Hamiltonian operator is discretized and represented as a matrix. This matrix (which is trigonal) acts on the discretized wave function to evolve it in time. The kinetic energy term has diagonal and off-diagonal contributions according to: 

$$ \frac{\text{d}^2 \psi}{\text{d}x^2} \approx \frac{\psi_{i+1} - 2\psi_i + \psi_{i-1}}{\Delta x^2}$$

While the potential energy only has diagonal terms. The descretized Hamiltonain matrix will take the form: 

$$ H =
\begin{pmatrix}
\frac{1}{\Delta x^2} + V(x_0) & -\frac{0.5}{\Delta x^2} & 0 & \cdots & 0 \\
-\frac{0.5}{\Delta x^2} & \frac{1}{\Delta x^2} + V(x_1) & -\frac{0.5}{\Delta x^2} & \cdots & 0 \\
0 & -\frac{0.5}{\Delta x^2} & \frac{1}{\Delta x^2} + V(x_2) & \cdots & 0 \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
0 & 0 & 0 & \cdots & \frac{1}{\Delta x^2} + V(x_{n-1})
\end{pmatrix}

$$

The Schrodinger equation: 

$$\frac{\partial \psi(x, t)}{\partial t} = -i \hat{H} \psi(x, t)$$ 

The Crank-Nicolson method approximates this equation at a time step $n$ and $n+1$ as follows [2]:

$$ \frac{\psi^{n+1} - \psi^n}{\Delta t} = -i \frac{\hat{H} \psi^{n+1} + \hat{H} \psi^n}{2} $$

Rearranging gives:

$$ \left( I + \frac{i \Delta t}{2} \hat{H} \right) \psi^{n+1} = \left( I - \frac{i \Delta t}{2} \hat{H} \right) \psi^n $$ 

Where $I$ is the identity matrix (representing the wave function without any Hamiltonian influence). If we set $A = \left( I + \frac{i \Delta t}{2} \hat{H} \right)$ and $ B =\left( I - \frac{i \Delta t}{2} \hat{H} \right) $ we get the final equation: 

$$ \psi^{n+1} = A^{-1} B \psi^n $$




In [5]:
class WaveAnimation:
    def __init__(self, wave_packet, max_steps):
        self.current_time = 0.0  # Initialize the time counter
        self.wave_sim = wave_packet  # Store the wave simulation object
        self.fig, self.ax = plt.subplots()  # Create a figure and axes for plotting

        # Add text for displaying time
        self.time_display = self.ax.text(0.05, 0.95, '', horizontalalignment='left', verticalalignment='top', transform=self.ax.transAxes)

        # Plot the potential barrier scaled by 0.1 for visibility
        plt.plot(self.wave_sim.x_values, self.wave_sim.potential_barrier * 0.1, color='r')

        # Plot the initial probability density of the wave packet
        self.prob_line, = self.ax.plot(self.wave_sim.x_values, self.wave_sim.evolve_wave_function())

        # Set plot limits and labels
        self.ax.set_ylim(0, 0.2)
        self.ax.set_xlabel('Position (a$_0$)') #position meausured in units of Bohr radius
        self.ax.set_ylabel('Probability Density')

        self.max_steps = max_steps  # Maximum number of time steps to run
        self.current_step = 0       # Current step counte

    def update_plot(self, new_data):
        # Update the plot with new probability density data
        self.prob_line.set_ydata(new_data) #set_ydata is a method of the Line2D class from matplotlib
        return self.prob_line,

    def time_generator(self):
        while self.current_step < self.max_steps:
            # Increment the elapsed time by the time step size of the wave packet
            self.current_time += self.wave_sim.time_step

            # Update the displayed time text with the current time in femtoseconds (unit conversion from atomic units to femtoseconds)
            self.time_display.set_text(f'Elapsed time: {self.current_time * 2.419e-2:.2f} fs')

            # Yield the updated probability density from the wave packet's evolution
            yield self.wave_sim.evolve_wave_function()
            self.current_step += 1
        plt.close(self.fig)  # Close the figure when max_steps is reached

    def start_animation(self):
        # Create the animation by repeatedly calling the update_plot method with new data
        self.animation = anim.FuncAnimation(self.fig, self.update_plot, self.time_generator, interval=5, blit=False, cache_frame_data=False) #based on FuncAnimation class from matplotlib, blit = False ensures that the entire plot is redrawn at each frame

The Transmission coefficient is given by the equation [2]: $$ T(E) = e^{-2 \sqrt{2m(V_0 - E)}\Delta x}  $$

where $m$ is the mass of the particle, $\Delta x$ is the barrier width, $V_0$ is the barrier height and $E$ is the energy of the particle.

### 1. Complete reflection, potential too wide and high

In [25]:
wave_packet = WavePacket(grid_size=500, time_step=5, potential_width=10, potential_height=1)
animator1 = WaveAnimation(wave_packet, max_steps=100)
animator1.start_animation()
plt.show()


In [ ]:
HTML(animator1.animation.to_html5_video())


### 2. Transmission occurs for a narrower potential

In [81]:
#narrow potential barrier
wave_packet = WavePacket(grid_size=500, time_step=5, potential_width=1, potential_height=1)
animator2 = WaveAnimation(wave_packet)
animator2.start_animation()
plt.show()

In [ ]:
HTML(animator2.animation.to_html5_video())

### 3. Transmission occurs however it is weaker

In [82]:
# lower potential barrier
wave_packet = WavePacket(grid_size=500, time_step=5, potential_width=10, potential_height=0.5)
animator3 = WaveAnimation(wave_packet)
animator3.start_animation()
plt.show()

In [ ]:
HTML(animator3.animation.to_html5_video())

### 4. Reflection happens along with transmission, clasically unexpected

In [83]:
# potential barrier less than energy of wave packet
wave_packet = WavePacket(grid_size=500, time_step=5, potential_width=10, potential_height=0.3)
animator4 = WaveAnimation(wave_packet)
animator4.start_animation()
plt.show()

In [ ]:
HTML(animator4.animation.to_html5_video())

The animations are in accordance with the equation for the trasmission coefficient, as decreasing the potnetial height or width results in a higher transmission coefficient. Overall we see that a wave packet could be transmitted or reflected even if its energy is lower or higher than the potential barrier.

### References:
[1] David Jeffrey Griffiths, et al. Introduction to Quantum Mechanics. 3rd ed., Cambridge, Cambridge University Press, 2018.

[2] “Python Implementation of Crank-Nicolson Scheme.” Marginalia, www.claudiobellei.com/2016/11/10/crank-nicolson/.